In [2]:
# !pip install pygame
import pygame
pygame.init()
screen = pygame.display.set_mode((800, 600))

In [ ]:
player1_controls = {
    "UP": pygame.K_w,
    "DOWN": pygame.K_s,
    "LEFT": pygame.K_a,
    "RIGHT": pygame.K_d,
    "ACTION": pygame.K_e
}

player2_controls = {
    "UP": pygame.K_UP,
    "DOWN": pygame.K_DOWN,
    "LEFT": pygame.K_LEFT,
    "RIGHT": pygame.K_RIGHT,
    "ACTION": pygame.K_SPACE
}


from Player import Player, Players
from Board import player1_start, player2_start
from constants import PLAYER1_GRAPHIC, PLAYER2_GRAPHIC
import os
os.chdir('..')

Player1 = Player(player1_start, PLAYER1_GRAPHIC, player1_controls)
Players.add(Player1)

Player2 = Player(player2_start, PLAYER2_GRAPHIC, player2_controls)
Players.add(Player2)

In [ ]:
from Objects import Walls, Floors, CBoards, Fryers
from Foods import Resources

def draw():
    screen.fill((255, 255, 255)) 
    Floors.draw(screen)
    Walls.draw(screen)
    Resources.draw(screen)
    Players.draw(screen)

In [ ]:
import numpy as np

def get_state():
    pixel_array = pygame.surfarray.array3d(screen)
    # Transpose to (height, width, channels)
    pixel_array = np.transpose(pixel_array, (1, 0, 2))
    
    #Normalize so that pixel values are from 0 to 1, not 0 to 255 as in RGB
    pixel_array = pixel_array / 255.0
    pixel_array = np.expand_dims(pixel_array, axis=0)

In [ ]:
time_per_game = int(input("Please enter the length of the game, in seconds (default: 180)"))
start_time = pygame.time.get_ticks()
from constants import START_X, END_X, START_Y, END_Y

def update():
    elapsed_time = (pygame.time.get_ticks() - start_time) / 1000
    if(elapsed_time < time_per_game):
        Players.update(pygame.key.get_pressed())
        CBoards.update()
        Fryers.update()


    else:
        #Game finished; display a large "GAME OVER" sign over a frozen frame
        font = pygame.font.SysFont("comicsansms", 100)
        game_over_surface = font.render("GAME OVER", True, (255, 0, 0))
        game_over_rect = game_over_surface.get_rect(center=((START_X+END_X)//2, (START_Y + END_Y)//2))
        screen.blit(game_over_surface, game_over_rect)

In [ ]:
from tensorflow.keras.models import load_model
import os
os.chdir('/Users/michal.maslowski/Documents/GitHub/OvercookedAI-coop')

Misha = load_model('Misha.h5') #Load the weights of the model

In [ ]:
from Foods import Menu
Menu = Menu()

pygame 2.6.0 (SDL 2.28.4, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


: 

In [ ]:
def get_statexdd():
    '''
    Returns the list of:
    - visual_data - normalized pixel values of the screen
    - numerical_data, containing in respective order:
        - the state of the Menu (see: Menu.get_state())
        - the state of Player 1's and Player 2's hands (see: Player.get_state())
    
    '''
    
    visual_data = pygame.surfarray.array3d(screen)
    visual_data = np.transpose(visual_data, (1, 0, 2)) #Change from width, height, color channel to height, width, color channel
    visual_data = visual_data / 255. #Normalize pixels from 0 to 1 for easier training

    numerical_data = np.concatenate((Menu.get_statecipa(), Player1.get_statexd(), Player2.get_statexd()))
    print("SHAPES: ", visual_data.shape, numerical_data.shape)    
    return [visual_data, numerical_data]

In [ ]:
running = True
clock = pygame.time.Clock()
tick = 0
game_score = 0


while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            break

    if(not running):
        break
    
    draw()
    update()
    
    pygame.display.flip()
    tick += 1
    clock.tick(60)
    
    if (tick % 100 == 0):
        xd = get_statexdd()
        print(xd)
    
        
pygame.display.quit()
pygame.quit()
exit()

None
None
None
None


KeyboardInterrupt: 

: 